In [1]:
from gurobipy import *
from math import *

In [2]:


q=((0,0),(0,40),(0,50),(1,49),(46,50),(48,48),(46,46),(50,48),(50,35),(30,5),(30,0)
   ,(50,50),(34,50),(34,9),(0,13),(25,25),(50,34),(34,22),(53,9),(30,13),(15,25)
  ,(22,50),(4,50),(5,9),(6,13),(4,25),(8,34),(8,22),(32,9),(8,13),(30,25));

V_h=90
V_c=18


qs=(0,0)
qf=(50,0)

In [3]:
qf[0]

50

In [4]:
# Create a new model
m = Model("MISOCP_2014")
n = 10

# Create variables
Tx = {};
Ty = {};
lx = {};
ly = {};
alpha = {};
f = {};
s = {};


#for SOCP
Cx= {};
Cy= {};
C= {};

d= {};

for i in range(1, n+1):
    Tx[i] = m.addVar(vtype = 'C',name='Tx[%s]'%i);
    Ty[i] = m.addVar(vtype = 'C',name='Ty[%s]'%i);
    lx[i] = m.addVar(vtype = 'C',name='lx[%s]'%i);
    ly[i] = m.addVar(vtype = 'C',name='ly[%s]'%i);
    for j in range(i, n+1):
        alpha[i,j] = m.addVar(lb=0,vtype = 'B',name='alpha[%s,%s]'%(i,j));
        f[i,j] = m.addVar(lb=0,vtype = 'C',name='f[%s,%s]'%(i,j));
        s[i,j] = m.addVar(lb=0,vtype = 'C',name='s[%s,%s]'%(i,j));
        d[i,j]=quicksum(sqrt( (q[k][0]-q[k+1][0])*(q[k][0]-q[k+1][0]) + (q[k][1]-q[k+1][1])*(q[k][1]-q[k+1][1]) ) for k in range(i,j));
        


            


        
# Integrate new variables

m.update()




Academic license - for non-commercial use only


In [5]:

# Add constraint: 
M =200
ctr=0;

for i in range(1, n+1):
    for j in range(i,n+1):
        
        #C18
        m.addConstr( f[i,j] - 0.35 <= M*(1-alpha[i,j])  ,'C.18.[%s,%s]'%(i,j));
        
        
        
        #C19a
        
        
        ctr=ctr+1;
    
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19a.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19a.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19a.[%s,%s]'%(i,j));
    
    
        m.addConstr( Cx[ctr]  == Tx[i]-lx[j], 'C19ax.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == Ty[i]-ly[j], 'C19ay.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19a.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*f[i,j])  == M*(1-alpha[i,j]), 'C19a.3.[%s,%s]'%(i,j));
        
    
        
        #C19b
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.1.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.1.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.1.[%s,%s]'%(i,j));
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.2.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.2.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.2.[%s,%s]'%(i,j));
        
        
        m.update()
        m.addConstr( Cx[ctr-1]  == Tx[i]-q[i][0], 'C19bx.1.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr-1]  == Ty[i]-q[i][1], 'C19by.1.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr-1]*Cx[ctr-1] + Cy[ctr-1]*Cy[ctr-1] <=  C[ctr-1]*C[ctr-1], 'C19b.2.1.[%s,%s]'%(i,j));
        
        
        m.addConstr( Cx[ctr]  == lx[j]-q[j][0], 'C19bx.1.2.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-q[j][1], 'C19by.1.2.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19b.2.2.[%s,%s]'%(i,j));
        
       

       
        
        m.addConstr( C[ctr] + d[i,j]+C[ctr-1]-V_h*f[i,j] == M*(1-alpha[i,j]), 'C.19b.3.[%s,%s]'%(i,j));
        
        
     
        
        
for i in range(1, n+1):
    for j in range(i,n):

        #C19c
      
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19c.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19c.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C19c.[%s,%s]'%(i,j));
            
        m.addConstr( Cx[ctr]  == lx[j]-Tx[j+1], 'C19cx.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-Ty[j+1], 'C19cy.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19c.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*s[i,j])  == M*(1-alpha[i,j]), 'C19c.3.[%s,%s]'%(i,j));
        
        

                 

#C2
for k in range(1, n+1):
    m.addConstr( quicksum(quicksum(alpha[i,j] for j in range(k,n+1)) for i in range(1,k+1))  == 1, 'C2.[%s]'%k); 
        
        


        



In [6]:
###Objective Function
a = m.addVar(lb=0,vtype = 'C',name='a');
b = m.addVar(lb=0,vtype = 'C',name='b');

ctr=ctr+1;
Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='nnn');


m.addConstr( Cx[ctr] == lx[n]-qf[0], 'obj_bx');



m.addQConstr( Tx[1]*Tx[1]+Ty[1]*Ty[1] <= a*a, 'obj_a');
m.addQConstr( Cx[ctr]*Cx[ctr] + ly[n]*ly[n] <= b*b, 'obj_b');


obj = (a+b)/V_c + quicksum(quicksum(f[i,j] for j in range(i,n+1)) for i in range(1,n+1) ) + quicksum(quicksum(s[i,j] for j in range(i,n)) for i in range(1,n+1)) 
m.setObjective(obj,GRB.MINIMIZE)

In [7]:
m.optimize()

Optimize a model with 641 rows, 838 columns and 1892 nonzeros
Model has 212 quadratic constraints
Variable types: 783 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 369 rows and 401 columns
Presolve time: 0.04s
Presolved: 272 rows, 437 columns, 804 nonzeros
Variable types: 415 continuous, 22 integer (22 binary)

Root relaxation: objective 0.000000e+00, 48 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8          -    0.00000      -     -    0s
     0     0    0.00000    0   11          -    0.00000      -     -    0s
H    0     0                       6.4162449    0.00000   100%     -    0s
*    0     0               0       6.4162

In [8]:

obj = m.getObjective()
print("t_m:",obj.getValue(),"hours\n")
print("M:",M,"\n")
ctr=0
for i in range(1,n+1):
    for j in range(i,n+1):
        if alpha[i,j].X >0.1:
            ctr=ctr+1
            print("Flyover no.", ctr, "from q",i, " to q",j,":\n" )
            print("T", i," : (",Tx[i].X,", ", Ty[i].X,")")
            print("l", j," : (",lx[j].X,", ", ly[j].X,")\n")

t_m: 6.248034955293704 hours

M: 200 

Flyover no. 1 from q 1  to q 1 :

T 1  : ( 6.377332214822178 ,  31.113047312798805 )
l 1  : ( 7.329914430409178 ,  35.88779513813707 )

Flyover no. 2 from q 2  to q 3 :

T 2  : ( 7.365485119695255 ,  36.06753834699945 )
l 3  : ( 12.216277579555681 ,  40.08745261199131 )

Flyover no. 3 from q 4  to q 6 :

T 4  : ( 33.206596764892154 ,  40.33671006336561 )
l 6  : ( 39.32010144325413 ,  38.815180504252986 )

Flyover no. 4 from q 7  to q 7 :

T 7  : ( 39.32010204079527 ,  38.81518006718392 )
l 7  : ( 41.38865542349201 ,  32.86446088451957 )

Flyover no. 5 from q 8  to q 8 :

T 8  : ( 41.42709573623922 ,  32.00640830565824 )
l 8  : ( 41.63134422934149 ,  27.490066414563795 )

Flyover no. 6 from q 9  to q 9 :

T 9  : ( 42.16322032930657 ,  15.823555238050243 )
l 9  : ( 42.43673474459472 ,  9.833924184642655 )

Flyover no. 7 from q 10  to q 10 :

T 10  : ( 42.444718117903754 ,  9.658415175685034 )
l 10  : ( 45.22869775507564 ,  4.0069225793393395 )



In [9]:
print(d[1,2])

<gurobi.LinExpr: 10.0>


In [10]:
m.getVars()

[<gurobi.Var Tx[1] (value 6.377332214822178)>,
 <gurobi.Var Ty[1] (value 31.113047312798805)>,
 <gurobi.Var lx[1] (value 7.329914430409178)>,
 <gurobi.Var ly[1] (value 35.88779513813707)>,
 <gurobi.Var alpha[1,1] (value 1.0)>,
 <gurobi.Var f[1,1] (value 0.27049197004761877)>,
 <gurobi.Var s[1,1] (value 0.010179987553856278)>,
 <gurobi.Var alpha[1,2] (value -0.0)>,
 <gurobi.Var f[1,2] (value 1.0791138471688555e-07)>,
 <gurobi.Var s[1,2] (value 3.909125112500078e-08)>,
 <gurobi.Var alpha[1,3] (value -0.0)>,
 <gurobi.Var f[1,3] (value 7.065611917767001e-08)>,
 <gurobi.Var s[1,3] (value 4.044129298108601e-08)>,
 <gurobi.Var alpha[1,4] (value 0.0)>,
 <gurobi.Var f[1,4] (value 1.2472999877388702e-07)>,
 <gurobi.Var s[1,4] (value 3.9598598178258726e-08)>,
 <gurobi.Var alpha[1,5] (value 0.0)>,
 <gurobi.Var f[1,5] (value 1.2466270327138318e-07)>,
 <gurobi.Var s[1,5] (value 3.9730077006083775e-08)>,
 <gurobi.Var alpha[1,6] (value 0.0)>,
 <gurobi.Var f[1,6] (value 7.162327442244987e-08)>,
 <gurob

In [11]:
f

{(1, 1): <gurobi.Var f[1,1] (value 0.27049197004761877)>,
 (1, 2): <gurobi.Var f[1,2] (value 1.0791138471688555e-07)>,
 (1, 3): <gurobi.Var f[1,3] (value 7.065611917767001e-08)>,
 (1, 4): <gurobi.Var f[1,4] (value 1.2472999877388702e-07)>,
 (1, 5): <gurobi.Var f[1,5] (value 1.2466270327138318e-07)>,
 (1, 6): <gurobi.Var f[1,6] (value 7.162327442244987e-08)>,
 (1, 7): <gurobi.Var f[1,7] (value 7.20668573706007e-08)>,
 (1, 8): <gurobi.Var f[1,8] (value 7.468898921558775e-08)>,
 (1, 9): <gurobi.Var f[1,9] (value 8.099374326775433e-08)>,
 (1, 10): <gurobi.Var f[1,10] (value 8.199975631839607e-08)>,
 (2, 2): <gurobi.Var f[2,2] (value 1.1021035817293523e-07)>,
 (2, 3): <gurobi.Var f[2,3] (value 0.35000004481459107)>,
 (2, 4): <gurobi.Var f[2,4] (value 1.1803902388862753e-07)>,
 (2, 5): <gurobi.Var f[2,5] (value 1.1173886882431816e-07)>,
 (2, 6): <gurobi.Var f[2,6] (value 6.866090629387145e-08)>,
 (2, 7): <gurobi.Var f[2,7] (value 6.889556480871306e-08)>,
 (2, 8): <gurobi.Var f[2,8] (value 7.

In [12]:
alpha

{(1, 1): <gurobi.Var alpha[1,1] (value 1.0)>,
 (1, 2): <gurobi.Var alpha[1,2] (value -0.0)>,
 (1, 3): <gurobi.Var alpha[1,3] (value -0.0)>,
 (1, 4): <gurobi.Var alpha[1,4] (value 0.0)>,
 (1, 5): <gurobi.Var alpha[1,5] (value 0.0)>,
 (1, 6): <gurobi.Var alpha[1,6] (value 0.0)>,
 (1, 7): <gurobi.Var alpha[1,7] (value 0.0)>,
 (1, 8): <gurobi.Var alpha[1,8] (value 0.0)>,
 (1, 9): <gurobi.Var alpha[1,9] (value 0.0)>,
 (1, 10): <gurobi.Var alpha[1,10] (value 0.0)>,
 (2, 2): <gurobi.Var alpha[2,2] (value 0.0)>,
 (2, 3): <gurobi.Var alpha[2,3] (value 1.0)>,
 (2, 4): <gurobi.Var alpha[2,4] (value 0.0)>,
 (2, 5): <gurobi.Var alpha[2,5] (value 0.0)>,
 (2, 6): <gurobi.Var alpha[2,6] (value 0.0)>,
 (2, 7): <gurobi.Var alpha[2,7] (value 0.0)>,
 (2, 8): <gurobi.Var alpha[2,8] (value 0.0)>,
 (2, 9): <gurobi.Var alpha[2,9] (value 0.0)>,
 (2, 10): <gurobi.Var alpha[2,10] (value 0.0)>,
 (3, 3): <gurobi.Var alpha[3,3] (value 0.0)>,
 (3, 4): <gurobi.Var alpha[3,4] (value 0.0)>,
 (3, 5): <gurobi.Var alpha[3

In [13]:
#m.write("out.lp")

In [14]:
#m.computeIIS()

In [15]:
status = m.status
if status == GRB.Status.OPTIMAL:
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj: %g' % m.objVal)
       
elif status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    # do IIS
    m.computeIIS()
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)

Tx[1] 6.37733
Ty[1] 31.113
lx[1] 7.32991
ly[1] 35.8878
alpha[1,1] 1
f[1,1] 0.270492
s[1,1] 0.01018
alpha[1,2] -0
f[1,2] 1.07911e-07
s[1,2] 3.90913e-08
alpha[1,3] -0
f[1,3] 7.06561e-08
s[1,3] 4.04413e-08
alpha[1,4] 0
f[1,4] 1.2473e-07
s[1,4] 3.95986e-08
alpha[1,5] 0
f[1,5] 1.24663e-07
s[1,5] 3.97301e-08
alpha[1,6] 0
f[1,6] 7.16233e-08
s[1,6] 3.97535e-08
alpha[1,7] 0
f[1,7] 7.20669e-08
s[1,7] 3.9864e-08
alpha[1,8] 0
f[1,8] 7.4689e-08
s[1,8] 4.02337e-08
alpha[1,9] 0
f[1,9] 8.09937e-08
s[1,9] 4.00982e-08
alpha[1,10] 0
f[1,10] 8.19998e-08
s[1,10] 0
Tx[2] 7.36549
Ty[2] 36.0675
lx[2] 115.082
ly[2] 125.412
alpha[2,2] 0
f[2,2] 1.1021e-07
s[2,2] 3.90912e-08
alpha[2,3] 1
f[2,3] 0.35
s[2,3] 1.16621
alpha[2,4] 0
f[2,4] 1.18039e-07
s[2,4] 3.95986e-08
alpha[2,5] 0
f[2,5] 1.11739e-07
s[2,5] 3.97301e-08
alpha[2,6] 0
f[2,6] 6.86609e-08
s[2,6] 3.97535e-08
alpha[2,7] 0
f[2,7] 6.88956e-08
s[2,7] 3.9864e-08
alpha[2,8] 0
f[2,8] 7.06126e-08
s[2,8] 4.02338e-08
alpha[2,9] 0
f[2,9] 7.56361e-08
s[2,9] 4.00961e-08

Cx.C19b.1.[4,9] -12.7934
Cy.C19b.1.[4,9] -9.66329
C.C19b.1.[4,9] 74.3988
Cx.C19b.2.[4,9] 12.4367
Cy.C19b.2.[4,9] 4.83392
C.C19b.2.[4,9] 66.4167
Cx.C19a.[4,10] -12.0221
Cy.C19a.[4,10] 36.3298
C.C19a.[4,10] 200
Cx.C19b.1.[4,10] -12.7934
Cy.C19b.1.[4,10] -9.66329
C.C19b.1.[4,10] 71.7931
Cx.C19b.2.[4,10] 15.2287
Cy.C19b.2.[4,10] 4.00692
C.C19b.2.[4,10] 64.0224
Cx.C19a.[5,5] 8.03053
Cy.C19a.[5,5] 12.113
C.C19a.[5,5] 200
Cx.C19b.1.[5,5] 61.982
Cy.C19b.1.[5,5] 64.0846
C.C19b.1.[5,5] 106.944
Cx.C19b.2.[5,5] 53.9514
Cy.C19b.2.[5,5] 51.9716
C.C19b.2.[5,5] 93.0563
Cx.C19a.[5,6] 70.6619
Cy.C19a.[5,6] 73.2694
C.C19a.[5,6] 200
Cx.C19b.1.[5,6] 61.982
Cy.C19b.1.[5,6] 64.0846
C.C19b.1.[5,6] 116.813
Cx.C19b.2.[5,6] -6.6799
Cy.C19b.2.[5,6] -7.18482
C.C19b.2.[5,6] 80.3584
Cx.C19a.[5,7] 68.5933
Cy.C19a.[5,7] 79.2201
C.C19a.[5,7] 200
Cx.C19b.1.[5,7] 61.982
Cy.C19b.1.[5,7] 64.0846
C.C19b.1.[5,7] 114.027
Cx.C19b.2.[5,7] -8.61134
Cy.C19b.2.[5,7] -15.1355
C.C19b.2.[5,7] 78.6726
Cx.C19a.[5,8] 68.3506
Cy.C19a.[5,

In [16]:
#m.write("SOCP_2014.lp")

In [17]:
#m.computeIIS()

In [18]:
obj = m.getObjective()
print(obj.getValue())

6.248034955293704


In [19]:
M

200